In [1]:
! docker run --name banquinhodb -d -e MYSQL_ROOT_PASSWORD=123456 -e MYSQL_DATABASE=banquinho -p 8080:3306 --rm  mysql:8.1.0


1dc9143df7fc56c049268f033336fef2ce174ec0c5c2b4ae6d127e297e0007c4


In [2]:
! pip install pandas pymysql

import pandas as pd
import warnings
import pymysql
warnings.filterwarnings('ignore')

Defaulting to user installation because normal site-packages is not writeable


In [3]:
def conexao():
    try:
        conn = pymysql.connect(
            user='root',
            host='127.0.0.1',
            port=8080,
            password='123456',
            database = "banquinho",
        )
        return conn
    except pymysql.Error as e:
        raise Exception('Erro ao se conectar: ' + str(e))

Não execute tudo, pois o banco leva ao menos 45segundos para iniciar o banco. 

In [6]:
conn = conexao()

In [7]:
from ddl_cliente_e_seus_relacionamentos import criar_tabela_tipo_cliente, criar_tabela_cliente
from ddl_produto_e_seus_relacionamentos import criar_tabelas_produtos
from ddl_cliente_produto import criar_tabela_cliente_produto
from dml_relacionamentos_produtos import inserir_diretores, inserir_categorias
from dml_produtos import inserir_produtos
from dml_clientes_seus_tipos import inserir_tipos_cliente, inserir_clientes



In [8]:
criar_tabela_tipo_cliente(conn)

In [9]:
criar_tabela_cliente(conn)

In [10]:
criar_tabelas_produtos(conn)

In [11]:
inserir_diretores(conn)

In [12]:
inserir_categorias(conn)

In [13]:
inserir_produtos(conn)

In [14]:
inserir_tipos_cliente(conn)

In [15]:
inserir_clientes(conn)

In [16]:
def listar_ids_e_precos_acima_media(conn):
    try:
        cursor = conn.cursor()

        dql_media_valor = """
        SELECT codigo_produto as ID, valor as Preço
        FROM produto
        WHERE valor > (SELECT AVG(valor) FROM produto);
        """
        cursor.execute(dql_media_valor)

        resultados = cursor.fetchall()

        # Converter os resultados em um DataFrame do Pandas
        df = pd.DataFrame(resultados, columns=['ID', 'Preço'])

        # Exibir o DataFrame com Pandas
        print(df)

    except pymysql.Error as e:
        raise Exception('Erro ao listar ids e preços acima da média: ', e)

In [17]:
listar_ids_e_precos_acima_media(conn)

   ID      Preço
0  13  100000.00
1  14  150000.00
2  19  300000.00


In [18]:
def listar_media_preco_por_categoria(conn):
    try:
        cursor = conn.cursor()

        dql_media_categoria = """
        SELECT categoria.tipo_produto AS Categoria, AVG(produto.valor) as PreçoMédio
        FROM categoria
        LEFT JOIN produto ON categoria.codigo_categoria = produto.codigo_categoria
        GROUP BY categoria.tipo_produto
        HAVING PreçoMédio IS NOT NULL
        ORDER BY categoria.tipo_produto;
        """
        cursor.execute(dql_media_categoria)

        resultados = cursor.fetchall()

        # Converter os resultados em um DataFrame do Pandas
        df = pd.DataFrame(resultados, columns=['Categoria', 'Preço Médio'])

        # Exibir o DataFrame com Pandas
        print(df)

    except pymysql.Error as e:
        raise Exception('Erro ao listar a média de preço por categoria: ', e)

In [19]:
listar_media_preco_por_categoria(conn)

                     Categoria    Preço Médio
0           Cartões de Crédito      50.000000
1               Conta Corrente       5.495000
2          Crédito Imobiliário  183333.333333
3         Empréstimos Pessoais    7500.000000
4             Internet Banking       0.000000
5  Investimentos em Renda Fixa    5333.333333
6       Pagamentos Eletrônicos       0.000000
7              Seguros de Vida      50.000000


In [20]:
def listar_produtos_mais_de_100(conn):
    try:
        cursor = conn.cursor()

        dql_produtos_caros = '''
        SELECT nome_produto AS Produto, valor AS Valor
        FROM produto
        WHERE valor > 100
        ORDER BY valor, nome_produto;
        '''
        cursor.execute(dql_produtos_caros)

        resultados = cursor.fetchall()

        # Converter os resultados em um DataFrame do Pandas
        df = pd.DataFrame(resultados, columns=['Produto', 'Valor'])

        # Exibir o DataFrame com Pandas
        print(df)
    except pymysql.Error as e:
        raise Exception('Erro ao listar produtos caros: ', e)

In [21]:
listar_produtos_mais_de_100(conn)

                        Produto      Valor
0               Seguro de Carro     200.00
1          Seguro de Carro Plus     500.00
2  Investimento em Renda Fixa A    1000.00
3         Empréstimo Pessoal P1    5000.00
4  Investimento em Renda Fixa B    5000.00
5         Empréstimo Pessoal P2   10000.00
6  Investimento em Renda Fixa C   10000.00
7         Crédito Imobiliário A  100000.00
8         Crédito Imobiliário B  150000.00
9         Crédito Imobiliário C  300000.00


In [22]:
! docker container stop banquinhodb

banquinhodb
